<a href="https://colab.research.google.com/github/Sylar257/Google-Cloud-Platform-with-Tensorflow/blob/master/Creating_dataset_with_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These code suppose to be run in a GCP instance. The instructions to set up such a instance
is documented in the *README* [file](https://github.com/Sylar257/Google-Cloud-Platform-with-Tensorflow/blob/master/READNE.md) of thie Repo.

## Creating data with SQL
In this notebook, we will be creating our dataset on a *natality* dataset from google *BigQuery*

In [0]:
# change these to try this notebook out
BUCKET = 'example_bucket_26_11'      # CHANGE this to a globally unique value. Your project name is a good option to try.
PROJECT = 'qwiklabs-gcp-00-09dd6f655043'     # CHANGE this to your project name
REGION = 'australia-southeast1-a'    # CHANGE this to one of the regions supported by Cloud AI Platform https://cloud.google.com/ml-engine/docs/tensorflow/regions

In [0]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# The following connects the BUCKET, PRIJECT and the workspace
# If BUCKET do not already exist, one will be created with default settings
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

### Loading the data
Use **SQL** query to access the natality data("LIMIT 1000"), and create a **`Pandas` dataframe** to contain our query data.<br>
The data is natality data (record of births in the US). My goal is to predict the baby's weight given a number of factors about the pregnancy and the baby's mother. Later, we will want to split the data into training and eval datasets. The hash of the year-month will be used for that -- this way, twins born on the same day won't end up in different cuts of the data.

In [1]:
# Folllowing three lines of code allows you to run it within colab as well
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [0]:
%load_ext google.colab.data_table

In [0]:
project_id = 'qwiklabs-gcp-00-b77bc3fac9b7'
from google.cloud import bigquery


In [11]:
import pandas as pd
# This is what we have from the last time
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  YEAR,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 2000
"""

df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,YEAR,hashmonth
0,7.063611,True,32,1,37.0,2001,7108882242435606404
1,4.687028,True,30,3,33.0,2001,7170969733900686954
2,7.561856,True,20,1,39.0,2001,6392072535155213407
3,7.561856,True,31,1,37.0,2001,2126480030009879160
4,7.312733,True,32,1,40.0,2001,3408502330831153141


#### `FARM_FINGERPRINT(value)`

This is a **Hash Function** in SQL that takes in a value(such as a *date* or a *string*) and computes the *fingerprint* of it(output format is **hashed numeric type**. *The output of this function for a particular input will never change*. Which means, `FARM_FINGERPRINT('26-jan-2012')` is always going to return the same value whenever we call.(and it’s different from calling on all other values)

In [17]:
# Let's create a query that all the babies born in the same month are put in the same bracket
# In addition, remove "LIMIT 2000" to have the entire dataset
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""


df = pd.io.gbq.read_gbq("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth",
                         project_id=project_id,
                         dialect='standard')
print("There are {} unique hashmonths between year 2000 and 2008.".format(len(df)))
df.head()

There are 96 unique hashmonths between year 2000 and 2008.


,hashmonth,num_babies
0,8904940584331855459,344191
1,2126480030009879160,344357
2,6691862025345277042,338820
3,5934265245228309013,324598
4,2995620979373137889,354257


#### `RAND() < value`
The `RAND()` will generate a random number between `0` and `1` each time, and we are only keeping the query data if the value is smaller than 0.01 which means it’s a 1% chance. Hence, we are getting 1% of the entire dataset.

In [26]:
# In addition, the entire dataset is shrinked to 0.5% of the total volume(randomly)
# Get 3/4 of the randomly shinked dataset into trainset.

trainQuery = "SELECT * FROM("+ query +") WHERE MOD(hashmonth,10)<8 AND RAND() < 0.0005"

# Get the rest of 1/4 of the randomly shinked dataset into evalset
evalQuery  = "SELECT * FROM("+ query +") WHERE MOD(hashmonth,10)>=8 AND RAND() < 0.0005"

# create training and eval dataframe
traindf = pd.io.gbq.read_gbq(trainQuery, project_id=project_id, dialect='standard')
evaldf  = pd.io.gbq.read_gbq(evalQuery, project_id=project_id, dialect='standard')

print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13485 examples in the train dataset and 2998 in the eval dataset


In [27]:
# Note that the ratio of len(trainset)/len(evalset) might not be exactly 8/2
# This is because of the mechanism of RAND()

13485/2998

4.497998665777184

## Data Preprocessing with Pandas
**Removing** `nan`s, column with invalid data and create a masked version data where we revome the information that has to be obtained from an ultrasound.<br>
This was because in the real case, the ultrasound information may or may not be available to our model. Hence, a robust ML model should
be able a good prediction with or without the ultrasound info.<br>
To tackle this, we duplicate our original dataframe and **mask** in `is_male` info and `plurality` info.

In [29]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.500126,False,19,1,40.0,774501970389208065
1,8.221038,False,22,1,35.0,774501970389208065
2,6.657960,False,25,1,40.0,774501970389208065
3,6.812284,True,32,2,36.0,774501970389208065
4,6.937947,False,35,1,39.0,774501970389208065


In [30]:
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13471.000000,13485.000000,13485.000000,13386.000000,1.348500e+04
mean,7.237199,27.404227,1.035595,38.600926,4.368422e+18
std,1.320609,6.163461,0.195415,2.554013,2.807565e+18
min,0.562179,13.000000,1.000000,18.000000,7.493147e+16
25%,6.563162,22.000000,1.000000,38.000000,1.639186e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,6.888635e+18
max,12.561940,49.000000,4.000000,47.000000,9.183606e+18


In [0]:
## Data cleaning and duplicate-mask preprocessing
def preprocess(df):
    # simply delete the columns where info is missing or invalid
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
    df['plurality'].replace(twins_etc, inplace=True)

    # Duplicate the oringinal dataframe and a mask on the info that requires an ultrasound
    masked = df.copy(deep=True) # deep copy means changes made to the two dataframe happen independently
    # well, without ultrasound at least we know if we have one baby or multiples
    masked.loc[masked['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
    masked['is_male'] = 'Unknown'

    return pd.concat([df, masked])

In [32]:
traindf_processed = preprocess(traindf)
evaldf_processed  = preprocess(evaldf)

# Note that the length of the processed is not exactly twice of the original
# Because invalid columns were removed
print(len(traindf), len(traindf_processed))
print(len(evaldf), len(evaldf_processed))

13485 26746
2998 5942


In [34]:
traindf_processed.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26746.000000,26746.000000,26746.000000,2.674600e+04
mean,7.238737,27.402901,38.607493,4.370578e+18
std,1.317821,6.152394,2.528207,2.805553e+18
min,0.562179,13.000000,19.000000,7.493147e+16
25%,6.563162,23.000000,38.000000,1.639186e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,6.888635e+18
max,12.561940,49.000000,47.000000,9.183606e+18


In [35]:
evaldf_processed.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,5942.000000,5942.000000,5942.000000,5.942000e+03
mean,7.199526,27.237630,38.609222,4.298721e+18
std,1.337825,6.214703,2.586416,2.730756e+18
min,0.676819,13.000000,18.000000,2.605984e+17
25%,6.563162,22.000000,38.000000,1.451354e+18
50%,7.312733,27.000000,39.000000,4.331750e+18
75%,8.062305,32.000000,40.000000,7.146494e+18
max,12.687603,49.000000,47.000000,8.904941e+18
